# Hands-on - Processamento de Linguagem Natural
### Prof. Feliphe Galiza

### Instruções para instalação dos pré-requisitos:

- conda create -n nlp python=3  
- conda activate nlp
- pip install jupyter spacy scikit-learn pandas matplotlib
- python -m spacy download pt_core_news_sm

Faça o download do dataset: [News of the Brazilian Newspaper](https://www.kaggle.com/marlesson/news-of-the-site-folhauol/downloads/news-of-the-site-folhauol.zip/2)

## Importando Ferramentas

In [1]:
# nlp
import spacy

# visualization
from spacy import displacy 

# data manipulation
import pandas as pd 
import string

# machine learning
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

# utils
import os
import gc

In [2]:
pd.options.mode.chained_assignment = None  # default='warn'

## Carregando o Dataset

In [3]:
os.listdir()

['.ipynb_checkpoints',
 'news-of-the-site-folhauol',
 '[NLP]-Analisando-Artigos-Folha-de-Sao-Paulo.ipynb']

In [4]:
path_csv = os.path.join('news-of-the-site-folhauol','articles.csv')

In [5]:
articles = pd.read_csv(path_csv)

## Explorando o Dataset

In [6]:
articles.head()

,title,text,date,category,subcategory,link
0,"Lula diz que está 'lascado', mas que ainda tem...",Com a possibilidade de uma condenação impedir ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
1,"'Decidi ser escrava das mulheres que sofrem', ...","Para Oumou Sangaré, cantora e ativista malines...",2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
2,Três reportagens da Folha ganham Prêmio Petrob...,Três reportagens da Folha foram vencedoras do ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
3,Filme 'Star Wars: Os Últimos Jedi' ganha trail...,A Disney divulgou na noite desta segunda-feira...,2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
4,CBSS inicia acordos com fintechs e quer 30% do...,"O CBSS, banco da holding Elopar dos sócios Bra...",2017-09-10,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/10/1...


In [7]:
articles.shape

(167053, 6)

In [8]:
articles['title'][3]

"Filme 'Star Wars: Os Últimos Jedi' ganha trailer definitivo; assista"

In [9]:
articles['text'][3]

'A Disney divulgou na noite desta segunda-feira (9) o novo trailer de "Star Wars: Os Últimos Jedi", oitavo episódio da saga.  O trailer era aguardado pelos fãs e se tornou um dos tópicos mais comentados no Twitter no horário de seu lançamento.  Assista ao trailer de \'Os Últimos Jedi\'  Assista ao trailer de \'Os Últimos Jedi\'  Em "O Despertar da Força" (2015), episódio mais recente, a personagem Rey (Daisy Ridley) descobre que tem a Força e procura por Luke Skywalker (Mark Hamill) para começar seu treinamento Jedi.  A história do novo episódio continua desse ponto, e cenas do trailer mostram a relação entre Rey e Skywalker.  Com direção de Rian Johnson, o filme será lançado em 14 de dezembro no Brasil. O nono episódio, ainda sem título, encerra a nova trilogia em 20 de dezembro de 2019.  O estúdio também divulgou novo poster do filme.  Poster'

## Extração de textos relevantes

Para análise de hoje utilizaremos apenas os títulos das notícias.

In [10]:
titles = articles['title']

In [11]:
del articles
gc.collect()

100

In [12]:
titles.head()

0    Lula diz que está 'lascado', mas que ainda tem...
1    'Decidi ser escrava das mulheres que sofrem', ...
2    Três reportagens da Folha ganham Prêmio Petrob...
3    Filme 'Star Wars: Os Últimos Jedi' ganha trail...
4    CBSS inicia acordos com fintechs e quer 30% do...
Name: title, dtype: object

In [13]:
titles.shape

(167053,)

## Carregando o modelo estatístico cmo suporte para a Língua Portuguesa

In [14]:
nlp = spacy.load('pt_core_news_sm')

## Tokenização
![The Token Object](https://course.spacy.io/doc.png)

In [15]:
test_text = titles[125]

In [16]:
print(test_text)

Com gol aos 50 min do 2º tempo, Egito vence e se classifica para a Copa-2018


In [17]:
doc = nlp(test_text)

# Create list of word tokens
token_list = []
for token in doc:
    token_list.append(token.text)
print(token_list)

['Com', 'gol', 'a', 'os', '50', 'min', 'do', '2º', 'tempo', ',', 'Egito', 'vence', 'e', 'se', 'classifica', 'para', 'a', 'Copa-2018']


## Remoção de Stop Words

In [18]:
pt_spacy_stopwords = spacy.lang.pt.stop_words.STOP_WORDS

In [19]:
print('Quantidade de stop words: %d' % len(pt_spacy_stopwords))
print('Primeiras 10 stop words: %s' % list(pt_spacy_stopwords)[:10])

Quantidade de stop words: 413
Primeiras 10 stop words: ['sete', 'ainda', 'fazes', 'mil', 'vossos', 'fez', 'zero', 'vão', 'nesta', 'ademais']


In [20]:
doc = nlp(test_text)

# filtering stop words
filtered_title = []
for word in doc:
    if word.is_stop==False:
        filtered_title.append(word)
print('Original Title: %s' % (test_text))
print("Filtered Title:",filtered_title)

Original Title: Com gol aos 50 min do 2º tempo, Egito vence e se classifica para a Copa-2018
Filtered Title: [gol, a, 50, min, 2º, ,, Egito, vence, e, classifica, a, Copa-2018]


### Atributos

In [21]:
print('Index:   ', [token.i for token in doc])
print('Text:    ', [token.text for token in doc])
print('is_alpha:', [token.is_alpha for token in doc])
print('is_punct:', [token.is_punct for token in doc])
print('like_num:', [token.like_num for token in doc])

Index:    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
Text:     ['Com', 'gol', 'a', 'os', '50', 'min', 'do', '2º', 'tempo', ',', 'Egito', 'vence', 'e', 'se', 'classifica', 'para', 'a', 'Copa-2018']
is_alpha: [True, True, True, True, False, True, True, False, True, False, True, True, True, True, True, True, True, False]
is_punct: [False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False]
like_num: [False, False, False, False, True, False, False, True, False, False, False, False, False, False, False, False, False, False]


## Lemmatização - Normalização Lexical

In [22]:
# finding lemma for each word
for word in doc:
    print(word.text,word.lemma_)

Com Com
gol gol
a o
os o
50 50
min min
do do
2º 2º
tempo tempo
, ,
Egito Egito
vence vencer
e e
se se
classifica classificar
para parir
a o
Copa-2018 Copa-2018


In [23]:
for word in nlp("química químicos químicas químico"):
    print(word.text,word.lemma_)

química químico
químicos químico
químicas químico
químico químico


In [24]:
for word in nlp("sou és é somos sois são era eras éramos fui fomos fostes seríamos"):
    print(word.text,word.lemma_)

sou ser
és ser
é ser
somos ser
sois ser
são ser
era ser
eras ser
éramos ser
fui ser
fomos ser
fostes ser
seríamos ser


In [25]:
doc = nlp(titles[5])
print('Text:    ', [token.text for token in doc])
print('Lemma:    ', [token.lemma_ for token in doc])

Text:     ['Em', 'encontro', ',', 'Bono', 'pergunta', 'a', 'Macri', 'sobre', 'argentino', 'desaparecido']
Lemma:     ['Em', 'encontrar', ',', 'Bono', 'perguntar', 'o', 'Macri', 'sobrar', 'argentino', 'desaparecer']


In [26]:
doc = nlp("Ontem, nós fomos até as escolas maiores escolas da região.")
print('Text:    ', [token.text for token in doc])
print('Lemma:    ', [token.lemma_ for token in doc])

Text:     ['Ontem', ',', 'nós', 'fomos', 'até', 'as', 'escolas', 'maiores', 'escolas', 'da', 'região', '.']
Lemma:     ['Ontem', ',', 'nó', 'ser', 'até', 'o', 'escola', 'maior', 'escola', 'da', 'região', '.']


# Part of Speech (POS) Tagging

In [27]:
for word in doc:
    print(word.text,word.pos_)

Ontem ADV
, PUNCT
nós PRON
fomos VERB
até ADP
as DET
escolas NOUN
maiores ADJ
escolas NOUN
da ADP
região NOUN
. PUNCT


In [28]:
displacy.render(doc, style="dep")

In [29]:
for word in nlp("João foi a escola com o seu irmão mais velho"):
    print(word.text,word.pos_)

João PROPN
foi VERB
a DET
escola NOUN
com ADP
o DET
seu DET
irmão NOUN
mais ADV
velho ADJ


In [30]:
displacy.render(nlp("João foi a escola com o seu irmão mais velho"), style="dep")

## Reconhecimento de Entidades Nomeadas (NER)

In [31]:
displacy.render(doc, style="ent")

c:\users\fgoncalx\appdata\local\continuum\miniconda3\envs\nlp\lib\runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


In [32]:
displacy.render(nlp(titles[10]), style="ent")

In [33]:
displacy.render(nlp(titles[148]), style="ent")

In [34]:
displacy.render(nlp(titles[158]), style="ent")

## Representação Vetorial

In [35]:
print(doc)

Ontem, nós fomos até as escolas maiores escolas da região.


In [36]:
print(doc.vector.shape)

(96,)


In [37]:
print(doc.vector)

[ 1.8306049   2.202352   -0.09417748 -0.9240098   0.5020909  -8.973923
 -2.6481946  -0.4182192   3.736682   -0.11602942  0.2978382   4.7163844
 -1.0272378  -1.1927911  -3.0771391  -2.9562864  -1.9975904  -1.6543967
 -3.7622802  -1.7333027  -0.8264892  -3.32527    -1.4468039  -1.207396
 -0.8728504   0.24511258 -2.2792506  -1.1265836   1.6327562   0.01705996
 -2.137542   -0.6468361   4.5818305   3.913627   -0.33186546 -0.562421
  0.8658557  -0.6165989   2.5860567   2.2439291   0.49272707  3.0552204
 -0.37686324 -0.8509813  -1.7606492  -0.9152525  -3.5854833  -0.10802865
  0.56886464 -2.1963565   1.7970386  -2.4235694  -4.4569383   0.97071
  0.28704572  0.4702338   4.500896    5.917587    3.0135543  -0.20570989
  7.3928256  -6.861406    0.21940355 -1.5986075   0.38154545 -3.105425
  2.8882608   2.1339772   5.918581   -3.506487    0.56533533 -4.229772
  5.1306677  -1.6032089  -7.2448      2.8124266   1.1603241   1.2935424
 -1.9907932  -6.597714    1.3922625  -1.0544064  -0.6971094   0.3615

# Categorização de Notícias

In [38]:
del titles
gc.collect()

27

In [39]:
articles = pd.read_csv(path_csv)

In [40]:
articles.head()

,title,text,date,category,subcategory,link
0,"Lula diz que está 'lascado', mas que ainda tem...",Com a possibilidade de uma condenação impedir ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
1,"'Decidi ser escrava das mulheres que sofrem', ...","Para Oumou Sangaré, cantora e ativista malines...",2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
2,Três reportagens da Folha ganham Prêmio Petrob...,Três reportagens da Folha foram vencedoras do ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
3,Filme 'Star Wars: Os Últimos Jedi' ganha trail...,A Disney divulgou na noite desta segunda-feira...,2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
4,CBSS inicia acordos com fintechs e quer 30% do...,"O CBSS, banco da holding Elopar dos sócios Bra...",2017-09-10,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/10/1...


In [41]:
dataset = articles[['title','category']]

In [42]:
del articles
gc.collect()

80

In [43]:
dataset = dataset[(dataset['category'] == 'mercado') | (dataset['category'] == 'esporte')]

In [44]:
dataset.shape

(40700, 2)

In [45]:
dataset.head()

,title,category
4,CBSS inicia acordos com fintechs e quer 30% do...,mercado
7,Tite diz querer seguir na seleção após o Mundi...,esporte
12,BNDES descarta irregularidades nos empréstimos...,mercado
13,Apple cita fornecedor francês antes de discuti...,mercado
14,"Volkswagen terá crescimento de 10% ao ano, diz...",mercado


In [46]:
# Create our list of punctuation marks
punctuations = string.punctuation

In [47]:
# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    
    # Cleaning text
    sentence = sentence.strip().lower()
    
    mytokens = nlp(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in pt_spacy_stopwords and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [48]:
spacy_tokenizer('Estamos aprendendo IA!')

['aprender']

# Machine Learning - Classificação de Textos

In [49]:
dataset['category'].unique()

array(['mercado', 'esporte'], dtype=object)

In [50]:
len(dataset['category'].unique())

2

In [51]:
le = preprocessing.LabelEncoder()

In [52]:
le.fit(dataset['category'])

LabelEncoder()

In [53]:
dataset['category_num'] = le.transform(dataset['category'])

In [54]:
dataset.head()

,title,category,category_num
4,CBSS inicia acordos com fintechs e quer 30% do...,mercado,1
7,Tite diz querer seguir na seleção após o Mundi...,esporte,0
12,BNDES descarta irregularidades nos empréstimos...,mercado,1
13,Apple cita fornecedor francês antes de discuti...,mercado,1
14,"Volkswagen terá crescimento de 10% ao ano, diz...",mercado,1


In [55]:
X = dataset['title']
y = dataset['category_num']

## TF-IDF

In [56]:
tfidf_vectorizer = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [57]:
X, _, y, _ = train_test_split(X, y, test_size=0.90) # shortcut to use only a few samples of the dataset

In [58]:
X_tfidf = tfidf_vectorizer.fit_transform(X)

In [59]:
X_tfidf.shape

(4070, 5627)

In [60]:
y.shape

(4070,)

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.30) # shortcut to use only a few samples of the dataset

In [62]:
X_train.shape

(2849, 5627)

In [63]:
y_train.shape

(2849,)

In [64]:
X_test.shape

(1221, 5627)

In [65]:
y_test.shape

(1221,)

In [66]:
classifier = RandomForestClassifier()

In [67]:
# training
classifier.fit(X_train,y_train)

c:\users\fgoncalx\appdata\local\continuum\miniconda3\envs\nlp\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [74]:
# Predicting
test_index=0
cat_list = ['esporte', 'mercado']
#print("Input: {}".format(X_tfidf[test_index]))
print("Input: {}".format(X.iloc[test_index]))
print("Model prediction: {}".format(cat_list[classifier.predict(X_tfidf[test_index])[0]]))
print("Ground Truth: {}".format(cat_list[y.iloc[test_index]]))

Input: Federação internacional mantém suspensão ao basquete brasileiro
Model prediction: esporte
Ground Truth: esporte


In [69]:
# Predicting
test_index=15
cat_list = ['esporte', 'mercado']
#print("Input: {}".format(X_tfidf[test_index]))
print("Input: {}".format(X.iloc[test_index]))
print("Model prediction: {}".format(cat_list[classifier.predict(X_tfidf[test_index])[0]]))
print("Ground Truth: {}".format(cat_list[y.iloc[test_index]]))

Input: Presidente da Siemens diz que crise política tem represado investimentos
Model prediction: mercado
Ground Truth: mercado


In [70]:
predicted = classifier.predict(X_tfidf)

In [71]:
# Model Accuracy
print("Model Accuracy:", metrics.accuracy_score(y, predicted))
print("Model F1-Score:", metrics.f1_score(y, predicted))

Model Accuracy: 0.9604422604422604
Model F1-Score: 0.9614186436616342
